<a href="https://colab.research.google.com/github/1804054Miraz/My_Thesis_Work/blob/main/EEG_features_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import glob

In [ ]:
alcoholic_path = "/content/drive/MyDrive/Dataset/EEG_FULL/New_alcoholic/"
alcoholic = []
csv_file_count = {}
i=0
for directory_name in os.listdir(alcoholic_path):
  directory_path = os.path.join(alcoholic_path, directory_name)

  if os.path.isdir(directory_path):
    csv_files = [file for file in os.listdir(directory_path) if file.endswith('.csv')]
    for file in csv_files:
      # print(directory_path+'/'+file)
      file_path = (directory_path+'/'+file)
      alcoholic.append(file_path)


In [ ]:
non_alcoholic_path = "/content/drive/MyDrive/Dataset/EEG_FULL/Transfer_new_non_alcoholic/"
non_alcoholic = []

for directory_name in os.listdir(non_alcoholic_path):
  directory_path = os.path.join(non_alcoholic_path, directory_name)

  if os.path.isdir(directory_path):
    csv_files = [file for file in os.listdir(directory_path) if file.endswith('.csv')]
    for file in csv_files:
      file_path = (directory_path+'/'+file)
      non_alcoholic.append(file_path)

In [ ]:
import numpy as np
import pandas as pd
import statistics as st
import scipy.stats as scst
from scipy.stats import skew
from scipy.stats import kurtosis
from scipy.stats import entropy
from scipy.stats import normaltest

In [ ]:
length = sum(1 for _ in alcoholic)
print("Length of the list:", length)

length = sum(1 for _ in non_alcoholic)
print("Length of the list:", length)


Length of the list: 3811
Length of the list: 3811


In [ ]:
import pandas as pd
import nolds
import chaospy as cp

In [ ]:
path = '/content/drive/MyDrive/Dataset/EEG_FULL/New_alcoholic/364/a_364_055_S2match.csv'
df = pd.read_csv(path)

In [ ]:
# import math
# import numpy as np
# import pandas as pd

# # Load EEG data from CSV
# path = '/content/drive/MyDrive/Dataset/EEG_FULL/New_alcoholic/364/a_364_055_S2match.csv'
# df = pd.read_csv(path)

# # Define your Cm function based on your specific requirements
# def Cm(data, m, r, i):
#     # For simplicity, let's use a basic rolling mean for illustration
#     window_data = data[i : i + m]
#     return np.mean(window_data)

# def calculate_ApEn(m, r, N, theta, data):
#     # Initialize the sum
#     sum_value = 0
#     epsilon = 1e-10  # Small constant to avoid taking the logarithm of zero

#     for i in range(N - m + 1):
#         # Calculate each term of the sum
#         term = math.log1p((Cm(data, m, r, i) + epsilon) / (N - m + 1)) + theta * (
#             r - math.log1p((Cm(data, m, r, i) + epsilon) / (N - m + 1))
#         )
#         sum_value += term

#     # Calculate ApEn
#     ApEn = (1 / (N - m + 1)) * sum_value
#     return ApEn

# # Example usage:
# m_value = 2  # embedding dimension
# r_value = 0.2  # radius
# N_value = len(df)  # length of the time series
# theta_value = 0.5

# # Dictionary to store results
# apen_results = {}

# # Loop through each electrode
# for column in df.columns:
#     electrode_data = df[column].values
#     apen_result = calculate_ApEn(m_value, r_value, N_value, theta_value, electrode_data)
#     apen_results[column] = apen_result

# # Convert the dictionary to a DataFrame if needed
# apen_df = pd.DataFrame(list(apen_results.items()), columns=['Electrode', 'ApEn'])

# # Display the DataFrame with ApEn values for each electrode
# print(apen_df)


In [ ]:
EEG_Features=pd.DataFrame(columns=['MAX', 'MIN', 'MEAN', 'STD', 'VAR', 'SKW', 'KUR', 'SE', 'LEE', 'PPA','RMS','NGE','ZC','label'])

In [ ]:
def find_SE(df):
  def shannon_entropy(column):
      # Convert the column to a numpy array
      values = np.array(column)

      # Ensure all values are non-negative
      values -= np.min(values)

      # Convert to integers
      values = values.astype(int)

      # Calculate the probability distribution of each unique value
      probabilities = np.bincount(values) / len(values)

      # Remove zero probabilities
      probabilities = probabilities[probabilities > 0]

      # Calculate Shannon entropy
      entropy = -np.sum(probabilities * np.log2(probabilities))

      return entropy
  # Calculate Shannon entropy for each column
  shannon_entropies = np.apply_along_axis(shannon_entropy, axis=0, arr=df.values)

  # Calculate overall Shannon entropy for all columns
  SE = np.sum(shannon_entropies)
  return SE

In [ ]:
def find_LEE(df):
  def log_energy_entropy(column):
      # Convert the column to a numpy array
      values = np.array(column)

      # Ensure all values are non-negative
      values -= np.min(values)

      # Add a small epsilon value to avoid division by zero
      epsilon = 1e-10
      values += epsilon

      # Calculate the Log Energy entropy
      energy = np.sum(values ** 2)
      log_energy_entropy = -np.sum((values ** 2 / energy) * np.log2(values ** 2 / energy))

      return log_energy_entropy

  # Calculate Log Energy entropy for each column
  log_energy_entropies = np.apply_along_axis(log_energy_entropy, axis=0, arr=df.values)

  # Calculate overall Log Energy entropy for all columns
  LEE = np.sum(log_energy_entropies)
  return LEE

In [ ]:
def find_rms(df):
  def calculate_column_rms(column):
      return np.sqrt(np.mean(column**2))

  # Calculate RMS value for each column
  column_rms_values = df.apply(calculate_column_rms)

  # Calculate RMS value of all the columns
  overall_rms_value = np.sqrt(np.mean(column_rms_values**2))
  return overall_rms_value

In [ ]:
%%capture
!pip install pyinform
import pandas as pd
import numpy as np
import pyinform
from scipy.stats import entropy

# Assuming df is your DataFrame
# Replace this with your actual DataFrame

# Function to calculate Negentropy for a given column
def find_ngentropy(df):
  # Function to calculate Negentropy for a given column
  def calculate_negentropy(column_data):
      prob_distribution = np.histogram(column_data, bins=256, range=(0, 256), density=True)[0]
      entropy_value = entropy(prob_distribution, base=2)
      negentropy = np.log2(2**entropy_value)
      return negentropy

  # Calculate overall Negentropy
  overall_negentropy = calculate_negentropy(df.values.flatten())

  # Print or display the final Negentropy DataFrame
  return overall_negentropy



In [ ]:
def find_zero(df):
# Function to find zero crossings for a given column
  def find_zero_crossings(column_data):
      zero_crossings = np.where(np.diff(np.sign(column_data)))[0]
      return len(zero_crossings)

  # Find zero crossings for each column
  zero_crossings_results = {}
  for column in df.columns:
      zero_crossings_results[column] = find_zero_crossings(df[column])

  # Calculate overall zero crossings
  overall_zero_crossings = sum(zero_crossings_results.values())
  return overall_zero_crossings

  # Print the overall zero crossings
  # print("Overall Zero Crossings:", overall_zero_crossings)


In [ ]:
def features_extraction(df, EEG_Features, i, val):
  add_values=[]
  # print()
  # EEG_Features['MAX'][j] = df.values.max()
  add_values.append(df.values.max())
  # print('MAX : ', MAX)

  add_values.append(df.values.min())
  # print('MIN : ', MIN)

  MEAN = np.mean(df, axis=0)
  add_values.append(np.mean(MEAN))
  # print('MEAN : ', MEAN)

  STD = np.std(df, axis=0)
  add_values.append(np.std(STD))
  # print('STD : ',STD)

  VAR = np.var(df, axis=0)
  add_values.append(np.var(VAR))
  # print('VAR : ', VAR)

  SKW = skew(df, axis=0)
  add_values.append(skew(SKW))
  # print('SKW : ', SKW)

  KUR = kurtosis(df, axis=0)
  add_values.append(kurtosis(KUR))
  # print('KUR : ', KUR)

  SE = find_SE(df)
  add_values.append(SE)
  # print('SHANON_ENTROPY : ', SE)

  LEE = find_LEE(df)
  add_values.append(LEE)
  # print('LOG_ENERGY_ENTROPY : ', LEE)

  # NORMALITY = df.apply(normaltest, axis=0)
  # NORMALITY = normaltest(NORMALITY)
  # print('NORMALITY : ', NORMALITY)
  #peak to peak amplitude
  PPA = np.ptp(df, axis=0)
  add_values.append(np.ptp(PPA))
  # print('PEAK_TO_PEAK_AMPLITUDE : ', PPA)

  RMS = find_rms(df)
  add_values.append(RMS)

  NGE = find_ngentropy(df)
  add_values.append(NGE)

  ZC = find_zero(df)
  add_values.append(ZC)

  add_values.append(val)
  # print(add_values)
  EEG_Features.loc[i] = add_values
  # EEG_Features = pd.concat([EEG_Features, EEG_Features.loc[0]], ignore_index=True)
  return EEG_Features

In [ ]:
# For alcoholic data
i=0
val=1.0
for path in alcoholic:
  # print(path)
  df=pd.read_csv(path)
  # print(df)
  EEG_Features = features_extraction(df, EEG_Features,i,val)
  i+=1
  # if i==100:
  #   break;

In [ ]:
# For Non-alcoholic data
val=0.0
for path in non_alcoholic:
  # print(path)
  df=pd.read_csv(path)
  # print(df)
  EEG_Features = features_extraction(df, EEG_Features,i,val)
  i+=1
  # if i==200:
  #   break;


In [ ]:
EEG_Features


,MAX,MIN,MEAN,STD,VAR,SKW,KUR,SE,LEE,PPA,RMS,NGE,ZC,label
0,42.826,-42.379,-2.133502,2.364078,878.795928,-0.972187,-0.039005,242.224337,486.902576,69.336,6.356042,3.405677,2011.0,1.0
1,78.573,-74.107,-5.455840,4.331395,8026.370564,0.722541,1.547945,263.171864,479.856202,98.145,10.260008,3.604806,1199.0,1.0
2,21.383,-67.413,-7.669460,2.320169,942.009385,-0.591528,-0.040461,256.546093,489.405860,78.125,10.021851,2.407722,923.0,1.0
3,79.356,-81.502,-5.391158,4.559711,9797.916632,0.429586,3.814154,269.395252,479.610238,100.098,10.465664,3.658152,1419.0,1.0
4,26.835,-59.835,-4.580334,2.461220,1097.870546,0.081715,-0.353628,266.733494,489.061640,69.336,8.510004,3.046801,1727.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7617,55.176,-99.548,-4.674924,6.046616,31778.109610,-0.088029,-1.168351,328.116727,487.646729,124.023,16.593290,4.781883,669.0,0.0
7618,42.084,-44.108,2.423221,2.903113,2427.901252,0.499885,-0.328437,299.485927,491.241884,69.336,10.161465,4.515614,889.0,0.0
7619,44.240,-66.406,0.690167,4.618951,14518.874109,0.239581,0.135848,317.144996,487.114770,89.355,11.482715,4.411233,898.0,0.0
7620,0.437,-0.051,0.000722,0.024173,0.000022,NaN,NaN,0.000000,509.672425,0.488,0.025040,0.000000,86.0,0.0


In [ ]:
EEG_Features.shape

(7622, 14)

In [ ]:
# EEG_Features.to_csv('/content/drive/MyDrive/Dataset/EEG_Features_FULL_again.csv', index=False)